In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [2]:
def normalizer(someseries):
    maxval = max(someseries)
    minval = min(someseries)
    newseries = (someseries - minval)/(maxval-minval)
    return newseries

def standardizer(someseries):
    mean = np.mean(someseries)
    stddev = np.std(someseries)
    someseries = (someseries - mean)/stddev
    return someseries

In [13]:
df = pd.read_csv('data/trace/salman.csv')
df = df.fillna("Nil")
df = df[df['Fog node'] != "Nil"]
#df = df[df['Distance'] != "undefined"]
df = df[df['Fog node'] != "test"]
temp = (df['Cost'].astype(float))
temp2 = standardizer(df['Distance'].astype(float))
df = df.drop(columns = ['Device','Time', 'Unnamed: 0', 'Day', 'Cost', 'Distance'])
df['Distance']  = temp2
df['Cost'] = normalizer(temp)
df = df.drop(columns = ['xcell', 'ycell'])
cols = list(df.columns)
cols = cols[2:len(cols)] + cols[0:2]
df = df[cols]

print("Shape of the dataframe is {}".format(df.shape))

Shape of the dataframe is (200073, 10)


In [14]:
prevvals = 10

inputrow = []
outputrow = []

for counter in range(0, len(df)-prevvals-1):
    temp = df.iloc[counter:counter+prevvals, 0:len(cols)-2]
    inputrow.append(np.array(temp))
    outputrow.append(df.iloc[counter+prevvals+1, len(cols)-3:len(df.columns)])

inputdf = np.array(inputrow)
outputdf = np.array(outputrow)

cutoff = int(0.80 * len(inputdf))
X_train = np.float32(inputdf[0:cutoff])
X_test = np.float32(inputdf[cutoff:len(inputdf)])
y_train = np.float32(outputdf[0:cutoff])
y_test = np.float32(outputdf[cutoff:len(outputdf)])

In [15]:
model = Sequential()
model.add(LSTM(50, input_shape=(inputdf.shape[1], inputdf.shape[2]), return_sequences=True))
model.add(LSTM(50))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(1))
model.compile(loss="mse", optimizer='adam', metrics=['mae'])
model.fit(X_train, y_train[:,0], epochs=10, batch_size=5000, verbose=True, validation_data=(X_test, y_test[:,0]))

Epoch 1/10
33/33 [==============================] - 19s 484ms/step - loss: 0.0130 - mae: 0.0876 - val_loss: 0.0052 - val_mae: 0.0559
Epoch 2/10
33/33 [==============================] - 14s 438ms/step - loss: 0.0058 - mae: 0.0558 - val_loss: 0.0048 - val_mae: 0.0520
Epoch 3/10
33/33 [==============================] - 14s 436ms/step - loss: 0.0056 - mae: 0.0546 - val_loss: 0.0048 - val_mae: 0.0510
Epoch 4/10
33/33 [==============================] - 15s 443ms/step - loss: 0.0055 - mae: 0.0541 - val_loss: 0.0046 - val_mae: 0.0507
Epoch 5/10
33/33 [==============================] - 14s 436ms/step - loss: 0.0054 - mae: 0.0536 - val_loss: 0.0045 - val_mae: 0.0504
Epoch 6/10
33/33 [==============================] - 14s 436ms/step - loss: 0.0052 - mae: 0.0532 - val_loss: 0.0044 - val_mae: 0.0505
Epoch 7/10
33/33 [==============================] - 14s 440ms/step - loss: 0.0051 - mae: 0.0528 - val_loss: 0.0050 - val_mae: 0.0567
Epoch 8/10
33/33 [==============================] - 15s 442ms/step - 

In [6]:
pred = model.predict(X_test)
print(pred[0:10])
# print(y_test[:, 0])
print(np.mean(np.abs((pred-y_test[:, 0])/pred)))

[[0.17910652]
 [0.1805083 ]
 [0.18027277]
 [0.18028952]
 [0.18140037]
 [0.18464465]
 [0.1868241 ]
 [0.19227536]
 [0.19636919]
 [0.19920154]]
